In [10]:
import pandas as pd
import numpy as np
import os
import datetime as dt

In [11]:
def TypeToDateTime(data):
    data['TIME'] = data['DATE'].astype("str").str.cat(data['TIME_M'])#Combine Date&Time to one column
    data['TIME'] = pd.to_datetime(data['TIME'], format = "%Y%m%d%H:%M:%S.%f")#Change type from string to datetime
    data = data.drop(columns = ['TIME_M','DATE']).set_index('TIME')
    return data

In [12]:
def QuoteInput(i,j):
    #path = os.path.abspath('TAQ')
    #data = pd.read_csv(path+'/'+i+'_quote/'+str(j)+'.csv')
    quote = pd.read_csv('/Users/yiyangqi/Documents/Spring 2020/Project/TAQ/'+i+'_quote/'+str(j)+'.csv')#import data
    quote.loc[(quote.ASK == 0)|(quote.BID == 0), ['ASK','BID']] = np.nan#Since 0 ask or bid price are outliers, turn them into nan
    quote = quote.dropna(subset=['BID','ASK'])#drop rows have 0 bid or ask price 
    quote = quote.loc[:,['BID','BIDSIZ','ASK','ASKSIZ','TIME_M','DATE']]#Only columns we need
    #quote['TradeLabel'] = 0
    return TypeToDateTime(quote)

In [13]:
def RawProcessing(data):
    data.loc[:,'TimeInt'] = data.index.astype('int')
    temp = data['TimeInt'].diff()
    temp = temp.replace(0,np.nan) #convert data has no time change with last data to 0
    temp = temp.fillna(method = 'ffill') #data has same time as last data, used data before last data
    temp = temp.fillna(100) #set first time change to be 0
    data.loc[:,'TimeChange'] = temp
    temp = None #clear out
    data.loc[data.TimeChange > 19800000000000, 'TimeChange'] = 100 #everyday is a new beginning 
    data = data.drop(columns = ['TimeInt'])
    data.loc[:,'Spread '] = data.loc[:,'ASK'] - data.loc[:,'BID']#new features we need in the model
    return data

In [14]:
def BidAskImbalance(data):
    data['BidAskImbalance'] = (data.loc[:,'BIDSIZ'] - data.loc[:,'ASKSIZ'])/(data.loc[:,'BIDSIZ'] + data.loc[:,'ASKSIZ'])#new features we need in the model
    return data                                                                        
                                                                             

In [15]:
def TWAP(data):
    #ser = pd.Series(["2019010210:00:00.000000000", "2019010215:30:00.000000000"] ) 
    #ser = pd.to_datetime(ser, format = "%Y%m%d%H:%M:%S.%f")
    #Trade_Time_Int = ser.astype('int')[1]-ser.astype('int')[0] #Figure out integer number of trading time in a day
    #Twap = dataa['TIME-Change']/dataa['TIME-int'].groupby(by = [dataa.index.day]).apply(lambda x: x-x[0])
    #Twap = data['TimeChange']/Trade_Time_Int
    
    data['TwapAsk'] = (data['TimeChange']*data['ASK']).groupby(by = [data.index.day]).cumsum()/data['TimeChange'].groupby(by = [data.index.day]).cumsum()
    #Calculated new features, we used sum of time change times price at that certain time point divided by sum of the time change till the same time
    data['TwapBid'] = (data['TimeChange']*data['BID']).groupby(by = [data.index.day]).cumsum()/data['TimeChange'].groupby(by = [data.index.day]).cumsum()
    return data

In [16]:
def VWAP(data):
    data['VwapAsk']=(data['ASKSIZ']*data['ASK']).groupby(by = [data.index.day]).cumsum()/data['ASKSIZ'].groupby(by = [data.index.day]).cumsum()
    #New features we used in model,we used sum of price*volume at that certain time point divided by sum of the volume till same time
    data['VwapBid']=(data['BIDSIZ']*data['BID']).groupby(by = [data.index.day]).cumsum()/data['BIDSIZ'].groupby(by = [data.index.day]).cumsum()
    return data

In [17]:
data = QuoteInput('TSLA',1)
data = data['2019/01/07':'2019/01/08']
data = RawProcessing(data)
data = BidAskImbalance(data)
data = TWAP(data)
data = VWAP(data)
data = (data-data.mean())/data.std()#standardize the data
data.to_hdf('TSLA0107to0108.h5', complib='zlib', complevel =9, key = 'Jan')

In [8]:
for j in range(1,13):
    data = QuoteInput('TSLA',j)
    data = RawProcessing(data)
    data = BidAskImbalance(data)
    data = TWAP(data)
    data = VWAP(data)
    if j == 1:
        temp = data
    else:
        frames = [temp,data]
        temp = pd.concat(frames)

#data = (temp-temp.min())/(temp.max()-temp.min())
#data.to_hdf('data.h5', complib='zlib', complevel =9, key = 'TSLA')

In [9]:
data = (temp-temp.mean())/temp.std()
data.to_hdf('TSLA.h5', complib='zlib', complevel =9, key = 'Jan')

In [ ]:
#Below are data we tried, but didn't fit our calculation power. They are way too big

In [17]:
def DIRECTION(data):
    #data['BidTemp'] = np.nan
    #data['AskTemp'] = np.nan
    for i in set(data.index.day):
        data.loc[(data.TradeLabel == 0)& (data.index.day == i) , 'BidTemp'] = data.loc[(data.TradeLabel == 0)& (data.index.day == i) , 'BID']
        data.loc[(data.TradeLabel == 0)& (data.index.day == i) , 'AskTemp'] = data.loc[(data.TradeLabel == 0)& (data.index.day == i) , 'ASK']
        data.loc[data.index.day == i , 'AskTemp'] = data.loc[data.index.day == i , 'AskTemp'].fillna(method='ffill',limit = 1)
        data.loc[data.index.day == i , 'BidTemp'] = data.loc[data.index.day == i , 'BidTemp'].fillna(method='ffill',limit = 1)
    data['Direction'] = data.loc[:,'BidTemp']+data.loc[:,'AskTemp']-2*data.loc[:,'BID']
    data.loc[:,'Direction'] = data.loc[:,'Direction'].fillna(0)
    data.loc[data.TradeLabel == 0, 'Direction'] = 0
    data.loc[data.Direction > 0, 'Direction'] = 1
    data.loc[data.Direction < 0, 'Direction'] = -1 #Close to Ask = 1, Close to Bid = -1
    return data
        

In [18]:
def MovingAverage(data):
    miu = data.rolling(1000).mean()
    re = 0
    for i in range(999):
        miu[i] = (data[i]+re)/(i+1)
        re += data[i]
    return miu

In [19]:
def BiasRatio(data):
    #data['TPTemp'] = np.nan
    #data['BidMovingAverage'] = np.nan
    #data['AskMovingAverage'] = np.nan
    for i in set(data.index.day):
        data.loc[(data.TradeLabel == 1)& (data.index.day == i) , 'TPTemp'] = data.loc[(data.TradeLabel == 1)& (data.index.day == i) , 'BID']
        data.loc[(data.index.day == i) , 'BidMovingAverage'] = MovingAverage(data.loc[(data.index.day == i) , 'BID'])
        data.loc[(data.index.day == i) , 'AskMovingAverage'] = MovingAverage(data.loc[(data.index.day == i) , 'ASK'])
        data.loc[data.index.day == i , 'TPTemp'] = data.loc[data.index.day == i , 'TPTemp'].fillna(method='ffill')
    data.loc[:,'TPTemp'] = data.loc[:,'TPTemp'].fillna(0)
    #BidMovingAverage = data['BID'].groupby(by = [data.index.day]).apply(MovingAverage)
    #AskMovingAverage = data['ASK'].groupby(by = [data.index.day]).apply(MovingAverage)
    data['BidBias'] = (data.loc[:,'TPTemp'] - data['BidMovingAverage'])/data['BidMovingAverage']
    data['AskBias'] = (data.loc[:,'TPTemp'] - data['AskMovingAverage'])/data['AskMovingAverage']
    data.loc[data.TPTemp == 0, ['AskBias','BidBias']] = 0
    BidMovingAverage = None
    AskMovingAverage = None
    return data

In [20]:
def DataProcessing(data):
    data = TWAP(data)
    data = BidAskRatio(data)
    data = VWAP(data)
    data = DIRECTION(data)
    data = BiasRatio(data)
    return data

In [21]:
def FinalProcessing(data):
    data.loc[:,['Direction','TradeLabel','TimeChange','ASKSIZ','BIDSIZ']] = data.loc[:, ['Direction','TradeLabel','TimeChange','ASKSIZ','BIDSIZ']].astype('int')
    #data = data.drop(['SYM_ROOT','BidMovingAverage','AskMovingAverage'], axis=1)
    data = data.reset_index()
    data = data.drop(['TIME','TimeInt', 'AskTemp','BidTemp','TPTemp','BidMovingAverage','AskMovingAverage'], axis=1)
    #data.loc[:,[elem for elem in list(data.columns) if elem not in ['TradeLabel','Direction','TimeChange','ASKSIZ','BIDSIZ']]] = data.loc[:,[elem for elem in list(data.columns) if elem not in ['TradeLabel','Direction','TimeChange','ASKSIZ','BIDSIZ']]].astype('float32')
    #data.loc[:,'BidAskRatio'] = data.loc[:,'BidAskRatio'].fillna(0)
    return data

In [13]:
def RawData(data1,data2):
    data = data1.append(data2, sort=False).sort_values(by = ['TIME'])
    data = data.drop('SYM_ROOT',axis=1)
    data['TimeInt'] = data.index.astype('int')
    temp = data['TimeInt'].diff()
    temp = temp.replace(0,np.nan) #convert data has no time change with last data to 0
    temp = temp.fillna(method = 'ffill') #data has same time as last data, used data before last data
    temp = temp.fillna(0) #set first time change to be 0
    data['TimeChange'] = temp
    temp = None #clear out
    data.loc[data.TimeChange > 19800000000000, 'TimeChange'] = 0 #everyday is a new beginning 
    return data

In [11]:
def TradeInput(i):
    #path = os.path.abspath('TAQ')
    #data = pd.read_csv(path+'/'+i+'_trade/'+i+'_trade.csv')
    data = pd.read_csv('/Users/yiyangqi/Documents/Spring 2020/Project/TAQ/'+i+'_trade/'+i+'_trade.csv')
    data['BID'] = data.loc[:,'PRICE']
    data['ASK'] = data.loc[:,'PRICE']
    data['BIDSIZ'] = data.loc[:,'SIZE']
    data['ASKSIZ'] = data.loc[:,'SIZE']
    data['TradeLabel'] = 1
    data = data.loc[:,['BID','BIDSIZ','ASK','ASKSIZ','SYM_ROOT','TIME_M','DATE','TradeLabel']]
    return TypeToDateTime(data)

In [22]:
for i in ['TSLA','AAPL']:
    trade = TradeInput(i)
    for j in range(1,13):
        if i == 'AAPL':
            trade_temp = trade['2019/'+str(j)]
        else:
            if j == 12:
                trade_temp = trade['2019/12/02':'2019/12/31']
            else:
                trade_temp = trade['2019/'+str(j)+'/2':'2019/'+str(j+1)+'/1']
        quote = QuoteInput(i,j)
        data = RawData(trade_temp,quote)
        data = DataProcessing(data)
        #data = data.drop(['TimeInt', 'AskTemp','BidTemp','TPTemp'], axis=1)
        data = FinalProcessing(data)
        if j  == 1:
            maximun = data.max()
            minimun = data.min()
        else:
            for k in maximun.index:
                if data.max()[k] > maximun[k]:
                    maximun[k] = data.max()[k]
                if data.min()[k] < minimun[k]:
                    minimun[k] = data.min()[k]
        data.to_csv(i+str(j)+'.csv')
        print('********************************************************')
    maximun.to_csv(i+'maximun'+'.csv')
    minimun.to_csv(i+'minimun'+'.csv')
    print('%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%')

********************************************************
********************************************************
********************************************************
********************************************************
********************************************************
********************************************************
********************************************************
********************************************************
********************************************************
********************************************************
********************************************************
********************************************************
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


/Users/yiyangqi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:27: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
/Users/yiyangqi/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:28: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.


********************************************************
********************************************************
********************************************************
********************************************************
********************************************************
********************************************************
********************************************************
********************************************************
********************************************************
********************************************************
********************************************************
********************************************************
%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%%


In [25]:
data.columns

Index(['Unnamed: 0', 'BID', 'BIDSIZ', 'ASK', 'ASKSIZ', 'TradeLabel',
       'TimeChange', 'TwapAsk', 'TwapBid', 'BidAskRatio', 'VwapAsk', 'VwapBid',
       'Direction', 'BidBias', 'AskBias'],
      dtype='object')

In [27]:
for i in ['TSLA','AAPL']:
    Max = pd.read_csv(i+'maximun.csv',header=None)
    Min = pd.read_csv(i+'minimun.csv',header=None)
    Max.columns = ['Index','Max']
    Min.columns = ['Index','Min']
    Max = Max.set_index('Index')
    Min = Min.set_index('Index')
    MaxMin = pd.concat([Max,Min],axis = 1)
    for j in range(1,13):
        data = pd.read_csv(i+str(j)+'.csv')
        data = data.drop('Unnamed: 0', axis = 1)
        for k in [elem for elem in list(data.columns) if elem not in ['TradeLabel','Direction','TimeChange']]:
            data.loc[:,k] = (data.loc[:,k]-MaxMin.loc[k,'Min'])/(MaxMin.loc[k,'Max'] - MaxMin.loc[k,'Min'])
        data.loc[:,[elem for elem in list(data.columns) if elem not in ['TradeLabel','Direction','TimeChange']]] = data.loc[:,[elem for elem in list(data.columns) if elem not in ['TradeLabel','Direction','TimeChange']]].astype('float32')
        data.loc[:,['Direction','TradeLabel','TimeChange']] = data.loc[:, ['Direction','TradeLabel','TimeChange']].astype('int')
        data.to_csv(i+str(j)+'.csv')
        print('^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^')
    print('快结束啦')

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
快结束啦
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [52]:
for i in ['TSLA','AAPL']:
    Max = pd.read_csv(i+'maximun.csv',header=None)
    Min = pd.read_csv(i+'minimun.csv',header=None)
    Max.columns = ['Index','Max']
    Min.columns = ['Index','Min']
    Max = Max.set_index('Index')
    Min = Min.set_index('Index')
    MaxMin = pd.concat([Max,Min],axis = 1)
    for j in range(1,13):
        data = pd.read_csv('/Users/yiyangqi/Documents/Spring 2020/Project/'+i+'/'+i+str(j)+'.csv')
        data = data.drop('Unnamed: 0', axis = 1)
        data.loc[:,'ASKSIZ'] = (data.loc[:,'ASKSIZ']-MaxMin.loc['ASKSIZ','Min'])/(MaxMin.loc['ASKSIZ','Max'] - MaxMin.loc['ASKSIZ','Min'])
        data.loc[:,'BIDSIZ'] = (data.loc[:,'BIDSIZ']-MaxMin.loc['BIDSIZ','Min'])/(MaxMin.loc['BIDSIZ','Max'] - MaxMin.loc['BIDSIZ','Min'])
        data.loc[:,[elem for elem in list(data.columns) if elem not in ['TradeLabel','Direction','TimeChange']]] = data.loc[:,[elem for elem in list(data.columns) if elem not in ['TradeLabel','Direction','TimeChange']]].astype('float32')
        data.loc[:,['Direction','TradeLabel','TimeChange']] = data.loc[:, ['Direction','TradeLabel','TimeChange']].astype('int')
        data.to_csv('/Users/yiyangqi/Documents/Spring 2020/Project/'+i+'/'+i+str(j)+'.csv')
        print('^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^')
    print('快结束啦')

^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
快结束啦
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [17]:
for i in ['TSLA','AAPL']:
    Max = pd.read_csv(i+'maximun.csv',header=None)
    Min = pd.read_csv(i+'minimun.csv',header=None)
    Max.columns = ['Index','Max']
    Min.columns = ['Index','Min']
    Max = Max.set_index('Index')
    Min = Min.set_index('Index')
    MaxMin = pd.concat([Max,Min],axis = 1)
    for j in range(1,13):
        data = pd.read_csv('/Users/yiyangqi/Documents/Spring 2020/Project/'+i+'_V2/'+i+str(j)+'.csv')
        data = data.drop('Unnamed: 0', axis = 1)
        data.loc[:,'TimeChange'] = (data.loc[:,'TimeChange']-MaxMin.loc['TimeChange','Min'])/(MaxMin.loc['TimeChange','Max'] - MaxMin.loc['TimeChange','Min'])
        data.loc[:,[elem for elem in list(data.columns) if elem not in ['TradeLabel','Direction']]] = data.loc[:,[elem for elem in list(data.columns) if elem not in ['TradeLabel','Direction']]].astype('float32')
        data.loc[:,['Direction','TradeLabel']] = data.loc[:, ['Direction','TradeLabel']].astype('int')
        data.to_hdf('/Users/yiyangqi/Documents/Spring 2020/Project/'+i+'.h5', key='month'+str(j),complevel=9, complib='blosc')
        print('|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||')
    print('**********************************************')

|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
**********************************************
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
|||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||||
|